In [4]:
%reload_ext autoreload
%autoreload 2

# Imports

In [5]:
from kret_notebook import *  # NOTE import first
from kret_matplotlib.mpl_nb_imports import *
from kret_np_pd.np_pd_nb_imports import *
from kret_polars.polars_nb_imports import *
from kret_rosetta.rosetta_nb_imports import *
from kret_sklearn.sklearn_nb_imports import *
from kret_torch_utils.torch_nb_imports import *
from kret_lightning.lightning_nb_imports import *
from kret_tqdm.tqdm_nb_imports import *
from kret_type_hints.types_nb_imports import *
from kret_utils.utils_nb_imports import *

# from kret_wandb.wandb_nb_imports import *  # NOTE this is slow to import

In [6]:
from projects.beijing.beijing_datamodule import BeijingDataModule
from projects.beijing.beijing_models import BeijingAirQualityLSTM
from projects.beijing.load_beijing_data import load_beijing_air_quality_data  # project_kretsinger

# Define Pipeline

In [7]:
float_cols = ["pm2.5", "year", "DEWP", "TEMP", "PRES", "Iws", "Is", "Ir"]
date_cols = ["month", "day", "hour"]
wind_cols = ["cbwd"]

In [8]:
date_time_normalizer = DateTimeSinCosNormalizer(
    datetime_cols={"month": 12, "day": 31, "hour": 24}
)  # Normalize 'month' and 'hour' columns
power_transformer = PowerTransformer(method="yeo-johnson", standardize=True)

wind_encoder = OrdinalEncoder()

column_transform = ColumnTransformer(
    transformers=[
        ("datetime", date_time_normalizer, date_cols),
        ("scaler", power_transformer, float_cols),
        ("windlabel", wind_encoder, wind_cols),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
    verbose=True,
)

In [9]:
pipeline_x = PipelinePD(steps=[("column_transform", column_transform)])
pipeline_y = PipelinePD(steps=[("scaler", power_transformer)])

# Load Data

In [10]:
beijing_dm = BeijingDataModule(UKS_PATHS.DATA_DIR / "beijing", 24, (pipeline_x, pipeline_y))
beijing_dm.post_init(batch_size=64, num_workers=4)

Saving hparams, ignoring ('pipeline_pd',)


# Implementation

## Instantiate model

In [11]:
# out =beijing_dm.data_preprocess()

In [12]:
beijing_dm.hparams_initial

"data_dir":        /Users/Akseldkw/coding/data_kretsinger/beijing
"sequence_length": 24
"split":           None

In [13]:
beijing_lstm = BeijingAirQualityLSTM(seq_length=beijing_dm.hparams_initial.sequence_length)
beijing_lstm.hparams_initial

TypeError: Can't instantiate abstract class BeijingAirQualityLSTM without an implementation for abstract method 'log_extra_metrics'

In [ ]:
logger = CSVLogger(**beijing_lstm.save_load_logging_dict)

In [ ]:
static_args = TrainerStaticDefaults.TRAINER_QUICK_ITER
static_args

{'min_epochs': 5,
 'max_epochs': 20,
 'check_val_every_n_epoch': 1,
 'log_every_n_steps': 10,
 'limit_val_batches': 0.1,
 'limit_test_batches': 0.1}

In [ ]:
static_args["max_epochs"] = 20

In [ ]:
dynamic_args = TrainerDynamicDefaults.trainer_dynamic_defaults(beijing_lstm, beijing_dm)
dynamic_args

{'logger': <lightning.pytorch.loggers.csv_logs.CSVLogger at 0x320653a40>,
 'default_root_dir': PosixPath('/Users/Akseldkw/coding/data_kretsinger/lightning_logs/BeijingAirQualityLSTM/v_000')}

In [ ]:
trainer_args = static_args | dynamic_args

In [ ]:
trainer = L.Trainer(**trainer_args)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


## Train Model

In [ ]:
raise ValueError("Stop here")  # STOP

ValueError: Stop here

In [ ]:
trainer.fit(model=beijing_lstm, datamodule=beijing_dm, ckpt_path=beijing_lstm.ckpt_file_name())

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory /Users/Akseldkw/coding/data_kretsinger/lightning_logs/BeijingAirQualityLSTM/v_000 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!


Setting up data for stage: TrainerFn.FITTING
Removed 2067 rows, representing 4.72% of the data
[ColumnTransformer] ...... (1 of 3) Processing datetime, total=   0.0s
[ColumnTransformer] ........ (2 of 3) Processing scaler, total=   0.1s
[ColumnTransformer] ..... (3 of 3) Processing windlabel, total=   0.0s
Setting up data for stage: validate
Removed 2067 rows, representing 4.72% of the data


/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /Users/Akseldkw/coding/data_kretsinger/lightning_logs/BeijingAirQualityLSTM/v_000 exists and is not empty.


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type       ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ wind_embedding │ Embedding  │     32 │ train │     0 │
│ 1 │ lstm           │ LSTM       │ 55.8 K │ train │     0 │
│ 2 │ model          │ Sequential │  6.3 K │ train │     0 │
└───┴────────────────┴────────────┴────────┴───────┴───────┘

Trainable params: 62.1 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 62.1 K                                                                                               
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 10                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

...

Metric val_loss improved. New best score: 1.469
Epoch 0, global step 522: 'val_loss' reached 1.46917 (best 1.46917), saving model to '/Users/Akseldkw/coding/data_kretsinger/lightning_logs/BeijingAirQualityLSTM/v_000/best-epoch=00-val_loss=1.47.ckpt' as top 1


Metric val_loss improved by 0.013 >= min_delta = 0.0001. New best score: 1.456
Epoch 1, global step 1044: 'val_loss' reached 1.45647 (best 1.45647), saving model to '/Users/Akseldkw/coding/data_kretsinger/lightning_logs/BeijingAirQualityLSTM/v_000/best-epoch=01-val_loss=1.46.ckpt' as top 1


Epoch 2, global step 1566: 'val_loss' was not in top 1


Epoch 3, global step 2088: 'val_loss' was not in top 1


Epoch 4, global step 2610: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


# Sandbox